In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

print("1. BAŞLADI: Kütüphaneler yüklendi, siteye gidiliyor...")

# Siteye bağlanalım
url = "http://books.toscrape.com/catalogue/category/books/science_22/index.html"
try:
    response = requests.get(url, timeout=10)
    print(f"2. BAĞLANTI DURUMU: {response.status_code} (200 ise başarılıdır)")
except:
    print("HATA: Siteye bağlanılamadı! İnternet bağlantını kontrol et.")

# İçeriği alalım
soup = BeautifulSoup(response.content, "html.parser")
urunler = soup.find_all("article", class_="product_pod")

print(f"3. VERİ DURUMU: Sayfada {len(urunler)} adet kitap bulundu.")

if len(urunler) == 0:
    print("SORUN BURADA: Siteye bağlandık ama kitapları bulamadık. HTML yapısı değişmiş olabilir.")
else:
    print("4. İŞLENİYOR: Veriler tabloya dökülüyor...")

    data = []
    for urun in urunler:
        baslik = urun.h3.a["title"]
        fiyat = urun.find("p", class_="price_color").text
        data.append({"Kitap": baslik, "Fiyat": fiyat})

    df = pd.DataFrame(data)

    print("\n--- İŞTE SONUÇLAR ---")
    print(df.head(5)) # İlk 5 tanesini göster
    print("---------------------")

1. BAŞLADI: Kütüphaneler yüklendi, siteye gidiliyor...
2. BAĞLANTI DURUMU: 200 (200 ise başarılıdır)
3. VERİ DURUMU: Sayfada 14 adet kitap bulundu.
4. İŞLENİYOR: Veriler tabloya dökülüyor...

--- İŞTE SONUÇLAR ---
                                               Kitap   Fiyat
0  The Most Perfect Thing: Inside (and Outside) a...  £42.96
1  Immunity: How Elie Metchnikoff Changed the Cou...  £57.36
2  Sorting the Beef from the Bull: The Science of...  £44.74
3  Tipping Point for Planet Earth: How Close Are ...  £37.55
4  The Fabric of the Cosmos: Space, Time, and the...  £55.91
---------------------


In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3

print("🚀 FİNAL PIPELINE BAŞLATILIYOR...")

# 1. VERİ ÇEKME (Scraping)
url = "http://books.toscrape.com/catalogue/category/books/science_22/index.html"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
urunler = soup.find_all("article", class_="product_pod")

data = []

for urun in urunler:
    baslik = urun.h3.a["title"]

    # KİRLİ VERİ: "£55.91"
    kirli_fiyat = urun.find("p", class_="price_color").text

    # TEMİZLİK İŞLEMİ (Data Cleaning)
    # 1. '£' işaretini kaldır
    # 2. 'Â' gibi görünmez karakterler varsa temizle
    # 3. Ondalıklı sayıya (float) çevir
    temiz_fiyat = float(kirli_fiyat.replace("£", "").replace("Â", ""))

    # Stok durumu
    stok = urun.find("p", class_="instock availability").text.strip()

    data.append({"Kitap_Adi": baslik, "Fiyat": temiz_fiyat, "Stok": stok})

# Tabloyu oluştur
df = pd.DataFrame(data)

# 2. SQL KAYDI
conn = sqlite3.connect("bilim_kitaplari.db")
df.to_sql("kitaplar_tablosu", conn, if_exists="replace", index=False)

# 3. EXCEL ÇIKTISI
df.to_excel("final_proje_verisi.xlsx", index=False)

print("\n✅ İŞLEM TAMAMLANDI!")
print(f"Toplam {len(df)} kitap verisi temizlendi ve SQL'e kaydedildi.")
print("Verilerin son haline bak:")
print(df.head()) # Artık fiyatlarda £ işareti olmadığını göreceksin

🚀 FİNAL PIPELINE BAŞLATILIYOR...

✅ İŞLEM TAMAMLANDI!
Toplam 14 kitap verisi temizlendi ve SQL'e kaydedildi.
Verilerin son haline bak:
                                           Kitap_Adi  Fiyat      Stok
0  The Most Perfect Thing: Inside (and Outside) a...  42.96  In stock
1  Immunity: How Elie Metchnikoff Changed the Cou...  57.36  In stock
2  Sorting the Beef from the Bull: The Science of...  44.74  In stock
3  Tipping Point for Planet Earth: How Close Are ...  37.55  In stock
4  The Fabric of the Cosmos: Space, Time, and the...  55.91  In stock
